In [19]:
from pathlib import Path
import subprocess

# -------------------------------------------------------
# 🎯 사용자 지정: 대상 영상 이름 (확장자 제외)
# -------------------------------------------------------
target_video = "M01_VISIT6_UpperLimb"

# -------------------------------------------------------
# 🧭 경로 설정
# -------------------------------------------------------
BASE_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/data/d02/Won_Kim_research_at_Bosanjin/M01")
OutputRoot = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/Won_Kim_research_at_Bosanjin/M01")

# 대상 영상 탐색
video_candidates = list(BASE_DIR.glob(f"{target_video}.*"))
if not video_candidates:
    raise FileNotFoundError(f"❌ 대상 영상이 없습니다: {target_video}")

Input = video_candidates[0]
OutputDir = OutputRoot / target_video
OutputDir.mkdir(parents=True, exist_ok=True)

print(f"""
📁 디렉토리 구조 설정 완료:
   • Input Video : {Input}
   • OutputDir   : {OutputDir}
""")

# -------------------------------------------------------
# ✂️ 세그먼트 정의 (frame 단위)
# -------------------------------------------------------
segments = [
    (13*30, 62*30, "frontal__biceps_curl__1"),
    (75*30, 127*30, "frontal__biceps_curl__2"),
    (140*30, 194*30, "frontal__biceps_curl__3"),
    (200*30, 254*30, "frontal__rowing__1"),
    (258*30, 315*30, "frontal__rowing__2"),
    (390*30, 460*30, "frontal__longsitting_knee_flexion__1"),
    (470*30, 760*30, "frontal__longsitting_knee_flexion__2"),
    ]

# -------------------------------------------------------
# 💾 segmentation txt 저장
# -------------------------------------------------------
segment_txt = OutputDir / "segment_frames.txt"
with open(segment_txt, "w", encoding="utf-8") as f:
    f.write("# label, start_frame, end_frame\n")
    for start, end, label in segments:
        f.write(f"{label}, {start}, {end}\n")

print(f"📝 segmentation info saved: {segment_txt}")

# -------------------------------------------------------
# ✂️ FFmpeg 컷팅 함수
# -------------------------------------------------------
def cut_video(input_video, start_frame, end_frame, output_video, fps=30):
    """FFmpeg으로 mp4 구간 자르기 (frame 기반, 정확/빠름/디버깅 포함)"""
    def to_ffmpeg_time(frame_idx):
        t = frame_idx / fps
        m, s = divmod(t, 60)
        return f"{int(m):02d}:{int(s):02d}"

    start = to_ffmpeg_time(start_frame)
    end = to_ffmpeg_time(end_frame)

    cmd = [
        "ffmpeg",
        "-y",
        "-ss", start,
        "-to", end,
        "-i", str(input_video),
        "-c", "copy",
        str(output_video)
    ]

    print(f"[DEBUG] 실행 명령어: {' '.join(cmd)}")
    proc = subprocess.run(cmd, capture_output=True, text=True)

    if proc.returncode != 0:
        print(f"❌ ffmpeg 오류:\n{proc.stderr}")
    else:
        print(f"✅ ffmpeg 성공적으로 실행됨")

    if Path(output_video).exists():
        size = Path(output_video).stat().st_size / 1024
        print(f"📁 파일 생성 확인: {output_video} ({size:.1f} KB)")
    else:
        print(f"⚠️ 파일이 생성되지 않았습니다: {output_video}")

# -------------------------------------------------------
# 🚀 컷팅 실행 (OutputDir에 직접 저장)
# -------------------------------------------------------
for start, end, label in segments:
    output_path = OutputDir / f"{label}.mp4"
    cut_video(Input, start, end, output_path)



📁 디렉토리 구조 설정 완료:
   • Input Video : /workspace/nas203/ds_RehabilitationMedicineData/data/d02/Won_Kim_research_at_Bosanjin/M01/M01_VISIT6_UpperLimb.MP4
   • OutputDir   : /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/Won_Kim_research_at_Bosanjin/M01/M01_VISIT6_UpperLimb

📝 segmentation info saved: /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/Won_Kim_research_at_Bosanjin/M01/M01_VISIT6_UpperLimb/segment_frames.txt
[DEBUG] 실행 명령어: ffmpeg -y -ss 00:01 -to 00:50 -i /workspace/nas203/ds_RehabilitationMedicineData/data/d02/Won_Kim_research_at_Bosanjin/M01/M01_VISIT6_UpperLimb.MP4 -c copy /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/Won_Kim_research_at_Bosanjin/M01/M01_VISIT6_UpperLimb/frontal__pec_dec_fly__1.mp4
✅ ffmpeg 성공적으로 실행됨
📁 파일 생성 확인: /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/Won_Kim_research_at_Bosanjin/M01/M01_VISIT6_UpperLimb/frontal__pec_dec_fly__1.mp4 (137548.1 KB)
[DEBUG] 실행 명령어: ffmpeg -y -ss 01:00 -to 01:55 -i 